# Исследование надежности заемщиков


Заказчик — кредитный отдел банка. 
Ему необходимо узнать:
+ влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Импорт библиотек

In [1]:
import pandas as pd
import math
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

### Загрузка данных

In [2]:
data = pd.read_csv('data.csv')

In [3]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Описание данных таблицы:
| Параметр                  | Описание                                             | Тип данных   | Наличие пропусков |
|---------------------------|------------------------------------------------------|--------------|-------------------|
| children                  | Количество детей в семье                             | Целое число  | нет       |
| days_employed             | Общий трудовой стаж в днях                           | Вещественное число | да  |
| dob_years                 | Возраст клиента в годах                              | Целое число  | нет       |
| education                 | Уровень образования клиента                          | Текст        | нет       |
| education_id              | Идентификатор уровня образования                     | Целое число  | нет       |
| family_status             | Семейное положение                                   | Текст        |нет        |
| family_status_id          | Идентификатор семейного положения                    | Целое число  | нет       |
| gender                    | Пол клиента                                          | Текст        | нет       |
| income_type               | Тип занятости                                        | Текст        | нет       |
| debt                      | Имел ли задолженность по возврату кредитов           | Целое число  | нет       |
| total_income              | Ежемесячный доход                                    | Вещественное число | да  |
| purpose                   | Цель получения кредита                               | Текст        | нет       |

## Предобработка данных

### Удаление пропусков

Ещё раз подтвердим наличие пропусков в полях таблицы days_employed и total_income

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Заполним пропуски в столбце total_income медианным значением в зависимости от типа дохода (должности)

In [6]:
for type_ in data['income_type'].unique():
    data.loc[(data['income_type'] == type_) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == type_), 'total_income'].median()

### Обработка аномальных значений

**Столбец days_employed**

In [7]:
data['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

На большей части данных у столбца трудового стажа, наблюдаются аномально низкие значения. Вероятно при загрузке данных произошла ошибка и числа стали отрицательные. Исправим данную аномалию взять значения столбца по модулю.

In [8]:
data['days_employed'] = data['days_employed'].abs()

Выведем медианное значение трудового стажа для каждого типа занятости.

In [9]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Характер таких аномалий неизвестен, поэтому решено оставить данные столбцы как есть, так как далее в анализе они использоваться не будут.

**Столбец children**

Выведем уникальные значения этого столбца

In [10]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце children есть два аномальных значения -1 и 20. Удалим строки с такими аномалиями из исходного датафрейма.

In [11]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Проверка корректности удаления

In [12]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

Заполним пропуски в столбце days_employed медианными значениями по каждого типа занятости income_type

In [13]:
for type_ in data['income_type'].unique():
    data.loc[(data['income_type'] == type_) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == type_), 'days_employed'].median()

Проверка корректности заполнения пропусков

In [14]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Изменим вещественный тип данных в столбце total_income на целочисленный

In [15]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Проверим данные на наличие неявных дубликатов

In [16]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Приведем все значения данного столбца к нижнему регистру

In [17]:
data['education'] = data['education'].str.lower()

Проверим данные на наличие полных дубликатов, если таковые есть удалим их

In [18]:
data.duplicated().sum()

71

In [19]:
data = data.drop_duplicates()

### Категоризация данных

**Столбец total_income**

На основании диапазонов, указанных ниже, сосздадим новый столбец total_income_category с категоризированным уровнем дохода.

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [20]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [21]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Проверим корректность создания нового столбца

In [22]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C


**Столбец purpose**

Выведем на экран перечень уникальных целей взятия кредита из столбца purpose

In [23]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Категоризируем данные на основании данных из этого столбца и создадим новый столбец purpose_category:

- операции с автомобилем,
- операции с недвижимостью,
- проведение свадьбы,
- получение образования.

In [24]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [25]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследование данных

**Определение зависимости между количеством детей и возвратом кредита в срок**

Сгруппируем данные клиентов по количеству детей и посчитаем сколько всего клиентов и сколько из них должников

In [26]:
сalc_debt_children = data.groupby('children').agg({'debt': ['sum','count']})
сalc_debt_children

debt       
           sum  count
children             
0         1063  14091
1          444   4808
2          194   2052
3           27    330
4            4     41
5            0      9

В таблице сalc_debt_children четко видно, что колличество информации о клиентах с бо'льшим числом детей, куда меньше чем тех, у кого детей нет вовсе. Подтвердим это подсчитав процент клиентов каждой категории от общего числа:

In [27]:
clients_sum_children = сalc_debt_children['debt']['count'].sum()
print('Общее колличество должников:', clients_sum_children)

Общее колличество должников: 21331


In [28]:
сalc_debt_children['category_to_all_%'] = сalc_debt_children['debt']['count'] / clients_sum_children * 100
сalc_debt_children['category_to_all_%'] = сalc_debt_children['category_to_all_%'].apply(math.ceil)
display(сalc_debt_children)

debt        category_to_all_%
           sum  count                  
children                               
0         1063  14091                67
1          444   4808                23
2          194   2052                10
3           27    330                 2
4            4     41                 1
5            0      9                 1

Здраво оценивать процент должников по таким данным мы не можем, поскольку в имеющихся данных не так много людей имеющих более 2ух детей.

Возьмем те категории клиентов на которые приходится по крайней мере 5% от общего числа. 

В таком случае останутся семьи с количеством детей от 0 до 2.

In [29]:
сalc_debt_children_filtered = сalc_debt_children.loc[сalc_debt_children['category_to_all_%']>=5]
сalc_debt_children_filtered

debt        category_to_all_%
           sum  count                  
children                               
0         1063  14091                67
1          444   4808                23
2          194   2052                10

Для более корректной оценки рассмотрим отношение должников к общему числу клиентов и отсортируем полученные значения для наглядности

In [30]:
сalc_debt_children_filtered['debt_to_total_%'] = \
    сalc_debt_children_filtered['debt']['sum']/сalc_debt_children_filtered['debt']['count']*100

сalc_debt_children_filtered['debt_to_total_%'] = round(сalc_debt_children_filtered['debt_to_total_%'],2)

сalc_debt_children_filtered = сalc_debt_children_filtered.sort_values(by = 'debt_to_total_%', ascending = False)
сalc_debt_children_filtered

debt        category_to_all_% debt_to_total_%
           sum  count                                  
children                                               
2          194   2052                10            9.45
1          444   4808                23            9.23
0         1063  14091                67            7.54

**Вывод:**
Согласно результатам последней таблицы, зависимость между количеством детей и наличием задолженности следующая: чем больше детей, тем выше риск задолженности

**Определение зависимости между семейным положением и возвратом кредита в срок**

Рассчитаем количество должников и общее число клиентов в каждой категории семейного положения

In [31]:
calc_debt_status = data.groupby('family_status').agg({'debt': ['sum','count']})
calc_debt_status

debt       
                       sum  count
family_status                    
Не женат / не замужем  273   2796
в разводе               84   1189
вдовец / вдова          63    951
гражданский брак       385   4134
женат / замужем        927  12261

Выведем процент клиентов каждой категории от общего числа

In [32]:
clients_sum_status = calc_debt_status['debt']['count'].sum()
print('Общее колличество должников:', clients_sum_status)

calc_debt_status['category_to_all_%'] = calc_debt_status['debt']['count'] / clients_sum_status * 100
calc_debt_status['category_to_all_%'] = calc_debt_status['category_to_all_%'].apply(math.ceil)
display(calc_debt_status)

Общее колличество должников: 21331


debt        category_to_all_%
                       sum  count                  
family_status                                      
Не женат / не замужем  273   2796                14
в разводе               84   1189                 6
вдовец / вдова          63    951                 5
гражданский брак       385   4134                20
женат / замужем        927  12261                58

Расcчитаем процент должников в каждой категории

In [33]:
calc_debt_status['debt_to_total_%'] = calc_debt_status['debt']['sum']/calc_debt_status['debt']['count']*100
calc_debt_status['debt_to_total_%'] = round(calc_debt_status['debt_to_total_%'],2)
calc_debt_status = calc_debt_status.sort_values(by = 'debt_to_total_%', ascending = False)
calc_debt_status

debt        category_to_all_% debt_to_total_%
                       sum  count                                  
family_status                                                      
Не женат / не замужем  273   2796                14            9.76
гражданский брак       385   4134                20            9.31
женат / замужем        927  12261                58            7.56
в разводе               84   1189                 6            7.06
вдовец / вдова          63    951                 5            6.62

**Вывод:** 

Рассмотрим рассчитанные данные парами:

+ _не женат/не замужем_ и _женат/замужем_ - количество должников среди клиентов сыгравших свадьбу меньше на целых два процента. Это логично, т.к. часто в семьях работают двое, соотвественно общий доход выше и выплачивать кредит легче.


+ _не женат/не замужем_ и _гражданский брак_ - колличество должников в этой категории практически одинаково. Скорее всего это связано с тем, что статусы могут пересекаться и среди не женатых могут находиться люди в гражданском браке.


+ _женат/замужем_ и _в разводе_ - эти категории отличаются друг от друга на половину процента в лучшую сторону. Интересно то, что процент уменьшается, хотя общий доход разделяется отдельно на каждого клиента. Возможно, это связано с тем, что клиенты брали кредит, когда находились в браке и дальше продолжают выплачивать его совместно. 

Самые лучшие показатели оказываются у категории вдовец/вдова, среди них должников всего около 6.6%

**Определение зависимости между уровнем дохода и возвратом кредита в срок**

Ранее мы разделяли доход на следующие категории:
- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

Рассчитаем количество общее количество клиентов и должников в каждой категории

In [34]:
calc_debt_income = data.groupby('total_income_category').agg({'debt': ['sum','count']})
calc_debt_income

debt       
                        sum  count
total_income_category             
A                         2     25
B                       354   5014
C                      1353  15921
D                        21    349
E                         2     22

Выведем процент клиентов каждой категории от общего числа

In [35]:
clients_sum_income = calc_debt_income['debt']['count'].sum()
print('Общее колличество должников:', clients_sum_income)

calc_debt_income['category_to_all_%'] = calc_debt_income['debt']['count'] / clients_sum_income * 100
calc_debt_income['category_to_all_%'] = calc_debt_income['category_to_all_%'].apply(math.ceil)
display(calc_debt_income)

Общее колличество должников: 21331


debt        category_to_all_%
                        sum  count                  
total_income_category                               
A                         2     25                 1
B                       354   5014                24
C                      1353  15921                75
D                        21    349                 2
E                         2     22                 1

Уберем нерелевантные категории

In [36]:
calc_debt_income_filtered = calc_debt_income.loc[calc_debt_income['category_to_all_%']>=5]
calc_debt_income_filtered

debt        category_to_all_%
                        sum  count                  
total_income_category                               
B                       354   5014                24
C                      1353  15921                75

Расcчитаем процент должников в каждой категории

In [37]:
calc_debt_income_filtered['debt_to_total_%'] = calc_debt_income_filtered['debt']['sum']/calc_debt_income_filtered['debt']['count']*100
calc_debt_income_filtered['debt_to_total_%'] = round(calc_debt_income_filtered['debt_to_total_%'],2)
calc_debt_income_filtered = calc_debt_income_filtered.sort_values(by = 'debt_to_total_%', ascending = False)
calc_debt_income_filtered

debt        category_to_all_% debt_to_total_%
                        sum  count                                  
total_income_category                                               
C                      1353  15921                75            8.50
B                       354   5014                24            7.06

**Вывод:**

Как и в предыдущих случаях из-за очень небольшого колличества данных по категориям с самыми низкими и с самыми высокими доходами тяжело сказать что-то о зависимости между уровнем дохода и наличием долга.

В данном случае можно более точно говорить только о категориях B и С, где вывод следующий: чем больше доход, тем меньше клиентов-должников среди них.

**Определение влияния целей кредита на возврат в срок**

Ранее мы определили следующие категории для цели кредита:

+ 'операции с автомобилем',
+ 'операции с недвижимостью',
+ 'проведение свадьбы',
+ 'получение образования'.

Аналогично, сгруппируем данные по категориям и рассчитаем сумарное количество клиентов и должников в каждой категории:

In [38]:
calc_debt_purpose = data.groupby('purpose_category').agg({'debt': ['sum','count']})
calc_debt_purpose

debt       
                          sum  count
purpose_category                    
операции с автомобилем    400   4279
операции с недвижимостью  780  10751
получение образования     369   3988
проведение свадьбы        183   2313

В данном случае, данных для оценки достаточно. Это покажет следующая отфильтрованная таблица:

In [39]:
clients_sum_purpose = calc_debt_purpose['debt']['count'].sum()
print('Общее колличество должников:', clients_sum_purpose)

calc_debt_purpose['category_to_all_%'] = calc_debt_purpose['debt']['count'] / clients_sum_purpose * 100
calc_debt_purpose['category_to_all_%'] = calc_debt_purpose['category_to_all_%'].apply(math.ceil)
calc_debt_purpose

Общее колличество должников: 21331


debt        category_to_all_%
                          sum  count                  
purpose_category                                      
операции с автомобилем    400   4279                21
операции с недвижимостью  780  10751                51
получение образования     369   3988                19
проведение свадьбы        183   2313                11

In [40]:
calc_debt_purpose_filtered = calc_debt_purpose.loc[calc_debt_purpose['category_to_all_%']>=5]
calc_debt_purpose_filtered

debt        category_to_all_%
                          sum  count                  
purpose_category                                      
операции с автомобилем    400   4279                21
операции с недвижимостью  780  10751                51
получение образования     369   3988                19
проведение свадьбы        183   2313                11

Расcчитаем процент должников в каждой категории

In [41]:
calc_debt_purpose_filtered['debt_to_total_%'] = \
    calc_debt_purpose_filtered['debt']['sum']/calc_debt_purpose_filtered['debt']['count']*100

calc_debt_purpose_filtered['debt_to_total_%'] = round(calc_debt_purpose_filtered['debt_to_total_%'],2)

calc_debt_purpose_filtered = calc_debt_purpose_filtered.sort_values(by = 'debt_to_total_%', ascending = False)
calc_debt_purpose_filtered

debt        category_to_all_% debt_to_total_%
                          sum  count                                  
purpose_category                                                      
операции с автомобилем    400   4279                21            9.35
получение образования     369   3988                19            9.25
проведение свадьбы        183   2313                11            7.91
операции с недвижимостью  780  10751                51            7.26

**Вывод:**

Как видно из последней таблицы наибольший процент должников, приходится на тех кто брал кредит на автомобиль и образование ~9%. В категориях проведение свадьбы и операции с недвижимостью процент должников ниже 7-8%.
Следовательно наличие задолженности зависит от цели получения кредита.

### Общий вывод.

Были рассмотрены данные клиентов, которые могли повлиять на наличие кредитной задолженности: количество детей, трудовой стаж, образование, семейный статус, пол, уровень дохода, тип дохода, цель получения кредита, наличие\осутствие задолженности.

**Данные имели следующие недочеты:**

+ Пропуски: в трудовом стаже и уровне дохода, поэтому было решено заполнить пропуски медианным значением в зависимости от типа дохода.
+ Аномальные значения: в столбце о количестве детей, строки с такими значениями были удалены.
+ Неявные дубликаты: устранены, приведением данных к общему виду.
+ Явные дубликаты были устранены, путем удаления соответвующих строк из таблицы.

**Данные были категорезированы:**

+ В зависимости от уровня дохода, были рассчитаны следующие категории:

0–30000 — 'E';

30001–50000 — 'D';

50001–200000 — 'C';

200001–1000000 — 'B';

1000001 и выше — 'A'.

+ В зависимости от цели получения кредита, данные были сгруппированы в следующие 4 категории:

'операции с автомобилем',

'операции с недвижимостью',

'проведение свадьбы',

'получение образования'.


**Были рассмотрены следующие гипотезы и сделаны соответствующие выводы:**

 
 + Есть ли зависимость между количеством детей и возвратом кредита в срок?
 
Зависимость следующая: с увеличением количества детей, риск невозврата кредита в срок возрастает и может доходить до 10%

 
 + Есть ли зависимость между семейным положением и возвратом кредита в срок?
 
В группе риска находятся клиенты в статусе _не женат/не замужем_ и _гражданский брак_, процент должников составляет около 9-10%

Более хорошие результаты показали клиенты в статусе _женат/замужем_ и _в разводе_, процент должников составляет около 7%

Наилучший результат показали клиенты в статусе вдовец/вдова - около 6.6%

 
 + Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
 
Зависимость следующая: чем выше уровень дохода, тем ниже риск задолженности.


 + Как разные цели кредита влияют на его возврат в срок?
 
В группе риска находятся клиенты взявшие кредит с категорией цели _операции с автомобилем_ и _оплата образования_, процент должников составляет около 9%.

Ниже риск имеют клиенты с категориями цели _проведение свадьбы_ и _операции с недвижимостью_ - 7-8%


**Таким образом** 

Идеальным заемщиком является клиент вдовец без детей с высоким уровнем дохода, который получил кредит с целью получения образования или покупки автомобиля

Клиентом с большим риском невыплаты кредита является человек не женатый или состоящий в гражданском браке с низким уровнем дохода, большим количеством детей, получивший кредит с целью приобретения недвижимости или организации свадьбы.